# Data Prep for Unsupervised Learning

## 1. Row Granularity

In [1]:
# create a sample dataframe
import pandas as pd

songs_dict = {
                'Customer': ['Aria', 'Aria', 'Aria', 'Chord', 'Chord', 'Harmony', 'Harmony', 'Harmony', 'Melody', 'Reed'],
                'Genre': ['Pop', 'Indie', 'Rock', 'Pop', 'Indie', 'Pop', 'Indie', 'Rock', 'Rock', 'Rock'],
                '# Songs': [50, 48, 1, 15, 36, 10, 5, 3, 2, 5]
             }

df = pd.DataFrame(songs_dict)
df

,Customer,Genre,# Songs
0,Aria,Pop,50
1,Aria,Indie,48
2,Aria,Rock,1
3,Chord,Pop,15
4,Chord,Indie,36
5,Harmony,Pop,10
6,Harmony,Indie,5
7,Harmony,Rock,3
8,Melody,Rock,2
9,Reed,Rock,5


### a. Group By

In [2]:
# use group by to sum up the songs for each customer
(df.groupby('Customer')['# Songs']
   .sum()
   .reset_index())

,Customer,# Songs
0,Aria,99
1,Chord,51
2,Harmony,18
3,Melody,2
4,Reed,5


In [3]:
# the original dataframe is unchanged
df.head()

,Customer,Genre,# Songs
0,Aria,Pop,50
1,Aria,Indie,48
2,Aria,Rock,1
3,Chord,Pop,15
4,Chord,Indie,36


In [4]:
# save the results as a new dataframe
customers_songs = (df.groupby('Customer')['# Songs']
                     .sum()
                     .reset_index())
customers_songs

,Customer,# Songs
0,Aria,99
1,Chord,51
2,Harmony,18
3,Melody,2
4,Reed,5


### b. Pivot

In [5]:
# view the dataframe again
df.head()

,Customer,Genre,# Songs
0,Aria,Pop,50
1,Aria,Indie,48
2,Aria,Rock,1
3,Chord,Pop,15
4,Chord,Indie,36


In [6]:
# use pivot to turn the genres into columns
(df.pivot(index='Customer',
          columns='Genre',
          values='# Songs'))

Genre,Indie,Pop,Rock
Customer,,,
Aria,48.0,50.0,1.0
Chord,36.0,15.0,NaN
Harmony,5.0,10.0,3.0
Melody,NaN,NaN,2.0
Reed,NaN,NaN,5.0


In [7]:
# add on a fillna to remove the NaN values
(df.pivot(index='Customer',
          columns='Genre',
          values='# Songs')
    .fillna(0))

Genre,Indie,Pop,Rock
Customer,,,
Aria,48.0,50.0,1.0
Chord,36.0,15.0,0.0
Harmony,5.0,10.0,3.0
Melody,0.0,0.0,2.0
Reed,0.0,0.0,5.0


In [8]:
# add on a reset_index to remove the multi-index
(df.pivot(index='Customer',
          columns='Genre',
          values='# Songs')
    .fillna(0)
    .reset_index())

Genre,Customer,Indie,Pop,Rock
0,Aria,48.0,50.0,1.0
1,Chord,36.0,15.0,0.0
2,Harmony,5.0,10.0,3.0
3,Melody,0.0,0.0,2.0
4,Reed,0.0,0.0,5.0


In [9]:
# save the results as a new dataframe
customers_genres = (df.pivot(index='Customer',
                             columns='Genre',
                             values='# Songs')
                      .fillna(0)
                      .reset_index())
customers_genres

Genre,Customer,Indie,Pop,Rock
0,Aria,48.0,50.0,1.0
1,Chord,36.0,15.0,0.0
2,Harmony,5.0,10.0,3.0
3,Melody,0.0,0.0,2.0
4,Reed,0.0,0.0,5.0


## 2. Non-Null and Numeric Columns

In [10]:
# read in the customers data
customers_raw = pd.read_csv('../Data/customers.csv')
customers_raw

,Name,Age,Followers,Income,Sign Up Date,Discount,Education Level
0,Aria,25.0,0.0,"$45,000",5/18/23,Yes,College
1,Chord,19.0,12.0,"$28,000",8/23/23,Yes,High School
2,Harmony,26.0,NaN,"$120,000",4/25/23,No,Graduate School
3,Melody,47.0,NaN,"$450,000",5/5/23,No,College
4,Reed,52.0,0.0,"$75,000",6/14/23,Yes,High School
5,Selena,NaN,1.0,"$62,000",8/26/23,No,College
6,Stefani,NaN,NaN,"$81,000",9/24/23,No,College
7,Taylor,33.0,52.0,"$60,000",9/8/23,No,High School


In [11]:
# create a dataframe that will hold clean data
customers = customers_raw.copy()
customers

,Name,Age,Followers,Income,Sign Up Date,Discount,Education Level
0,Aria,25.0,0.0,"$45,000",5/18/23,Yes,College
1,Chord,19.0,12.0,"$28,000",8/23/23,Yes,High School
2,Harmony,26.0,NaN,"$120,000",4/25/23,No,Graduate School
3,Melody,47.0,NaN,"$450,000",5/5/23,No,College
4,Reed,52.0,0.0,"$75,000",6/14/23,Yes,High School
5,Selena,NaN,1.0,"$62,000",8/26/23,No,College
6,Stefani,NaN,NaN,"$81,000",9/24/23,No,College
7,Taylor,33.0,52.0,"$60,000",9/8/23,No,High School


### a. Handling Null Values

In [12]:
# null values in each column
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             8 non-null      object 
 1   Age              6 non-null      float64
 2   Followers        5 non-null      float64
 3   Income           8 non-null      object 
 4   Sign Up Date     8 non-null      object 
 5   Discount         8 non-null      object 
 6   Education Level  8 non-null      object 
dtypes: float64(2), object(5)
memory usage: 580.0+ bytes


In [13]:
# cells with null values
customers.isna()

,Name,Age,Followers,Income,Sign Up Date,Discount,Education Level
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False
3,False,False,True,False,False,False,False
4,False,False,False,False,False,False,False
5,False,True,False,False,False,False,False
6,False,True,True,False,False,False,False
7,False,False,False,False,False,False,False


In [14]:
# rows with null values
customers.isna().any(axis=1)

0    False
1    False
2     True
3     True
4    False
5     True
6     True
7    False
dtype: bool

In [15]:
# actual rows with null values
customers[customers.isna().any(axis=1)]

,Name,Age,Followers,Income,Sign Up Date,Discount,Education Level
2,Harmony,26.0,NaN,"$120,000",4/25/23,No,Graduate School
3,Melody,47.0,NaN,"$450,000",5/5/23,No,College
5,Selena,NaN,1.0,"$62,000",8/26/23,No,College
6,Stefani,NaN,NaN,"$81,000",9/24/23,No,College


#### i. Dropping Missing Values

In [16]:
# original data frame
customers

,Name,Age,Followers,Income,Sign Up Date,Discount,Education Level
0,Aria,25.0,0.0,"$45,000",5/18/23,Yes,College
1,Chord,19.0,12.0,"$28,000",8/23/23,Yes,High School
2,Harmony,26.0,NaN,"$120,000",4/25/23,No,Graduate School
3,Melody,47.0,NaN,"$450,000",5/5/23,No,College
4,Reed,52.0,0.0,"$75,000",6/14/23,Yes,High School
5,Selena,NaN,1.0,"$62,000",8/26/23,No,College
6,Stefani,NaN,NaN,"$81,000",9/24/23,No,College
7,Taylor,33.0,52.0,"$60,000",9/8/23,No,High School


In [17]:
# drop rows with null values
customers.dropna()

,Name,Age,Followers,Income,Sign Up Date,Discount,Education Level
0,Aria,25.0,0.0,"$45,000",5/18/23,Yes,College
1,Chord,19.0,12.0,"$28,000",8/23/23,Yes,High School
4,Reed,52.0,0.0,"$75,000",6/14/23,Yes,High School
7,Taylor,33.0,52.0,"$60,000",9/8/23,No,High School


In [18]:
# optional: can reset the index
customers.dropna().reset_index()

,index,Name,Age,Followers,Income,Sign Up Date,Discount,Education Level
0,0,Aria,25.0,0.0,"$45,000",5/18/23,Yes,College
1,1,Chord,19.0,12.0,"$28,000",8/23/23,Yes,High School
2,4,Reed,52.0,0.0,"$75,000",6/14/23,Yes,High School
3,7,Taylor,33.0,52.0,"$60,000",9/8/23,No,High School


In [19]:
# and also remove the extra column
customers.dropna().reset_index(drop=True)

,Name,Age,Followers,Income,Sign Up Date,Discount,Education Level
0,Aria,25.0,0.0,"$45,000",5/18/23,Yes,College
1,Chord,19.0,12.0,"$28,000",8/23/23,Yes,High School
2,Reed,52.0,0.0,"$75,000",6/14/23,Yes,High School
3,Taylor,33.0,52.0,"$60,000",9/8/23,No,High School


In [20]:
# the rows are still there
customers

,Name,Age,Followers,Income,Sign Up Date,Discount,Education Level
0,Aria,25.0,0.0,"$45,000",5/18/23,Yes,College
1,Chord,19.0,12.0,"$28,000",8/23/23,Yes,High School
2,Harmony,26.0,NaN,"$120,000",4/25/23,No,Graduate School
3,Melody,47.0,NaN,"$450,000",5/5/23,No,College
4,Reed,52.0,0.0,"$75,000",6/14/23,Yes,High School
5,Selena,NaN,1.0,"$62,000",8/26/23,No,College
6,Stefani,NaN,NaN,"$81,000",9/24/23,No,College
7,Taylor,33.0,52.0,"$60,000",9/8/23,No,High School


In [21]:
# save to new dataframe to keep the updates
customers_dropped_rows = customers.dropna().reset_index(drop=True)
customers_dropped_rows

,Name,Age,Followers,Income,Sign Up Date,Discount,Education Level
0,Aria,25.0,0.0,"$45,000",5/18/23,Yes,College
1,Chord,19.0,12.0,"$28,000",8/23/23,Yes,High School
2,Reed,52.0,0.0,"$75,000",6/14/23,Yes,High School
3,Taylor,33.0,52.0,"$60,000",9/8/23,No,High School


#### ii. Imputing Missing Values

In [22]:
# the original dataframe
customers

,Name,Age,Followers,Income,Sign Up Date,Discount,Education Level
0,Aria,25.0,0.0,"$45,000",5/18/23,Yes,College
1,Chord,19.0,12.0,"$28,000",8/23/23,Yes,High School
2,Harmony,26.0,NaN,"$120,000",4/25/23,No,Graduate School
3,Melody,47.0,NaN,"$450,000",5/5/23,No,College
4,Reed,52.0,0.0,"$75,000",6/14/23,Yes,High School
5,Selena,NaN,1.0,"$62,000",8/26/23,No,College
6,Stefani,NaN,NaN,"$81,000",9/24/23,No,College
7,Taylor,33.0,52.0,"$60,000",9/8/23,No,High School


In [23]:
# view the columns with null values
customers[['Age', 'Followers']]

,Age,Followers
0,25.0,0.0
1,19.0,12.0
2,26.0,NaN
3,47.0,NaN
4,52.0,0.0
5,NaN,1.0
6,NaN,NaN
7,33.0,52.0


In [24]:
# impute with the median of a column
customers.Age.fillna(customers.Age.median())

0    25.0
1    19.0
2    26.0
3    47.0
4    52.0
5    29.5
6    29.5
7    33.0
Name: Age, dtype: float64

In [25]:
# round the age
round(customers.Age.fillna(customers.Age.median()))

0    25.0
1    19.0
2    26.0
3    47.0
4    52.0
5    30.0
6    30.0
7    33.0
Name: Age, dtype: float64

In [26]:
# impute with domain expertise
customers.Followers.fillna(0)

0     0.0
1    12.0
2     0.0
3     0.0
4     0.0
5     1.0
6     0.0
7    52.0
Name: Followers, dtype: float64

In [27]:
# save the updates to the dataframe
customers.Age = round(customers.Age.fillna(customers.Age.median()))
customers.Followers = customers.Followers.fillna(0)

In [28]:
# view the cleaned up columns
customers[['Age', 'Followers']]

,Age,Followers
0,25.0,0.0
1,19.0,12.0
2,26.0,0.0
3,47.0,0.0
4,52.0,0.0
5,30.0,1.0
6,30.0,0.0
7,33.0,52.0


### b. Converting Data Types

In [29]:
# view the dataframe
customers

,Name,Age,Followers,Income,Sign Up Date,Discount,Education Level
0,Aria,25.0,0.0,"$45,000",5/18/23,Yes,College
1,Chord,19.0,12.0,"$28,000",8/23/23,Yes,High School
2,Harmony,26.0,0.0,"$120,000",4/25/23,No,Graduate School
3,Melody,47.0,0.0,"$450,000",5/5/23,No,College
4,Reed,52.0,0.0,"$75,000",6/14/23,Yes,High School
5,Selena,30.0,1.0,"$62,000",8/26/23,No,College
6,Stefani,30.0,0.0,"$81,000",9/24/23,No,College
7,Taylor,33.0,52.0,"$60,000",9/8/23,No,High School


In [30]:
# data type of each column
customers.dtypes

Name                object
Age                float64
Followers          float64
Income              object
Sign Up Date        object
Discount            object
Education Level     object
dtype: object

#### i. Converting to Numeric

In [31]:
# income was read in as a text field
customers[['Income']]

,Income
0,"$45,000"
1,"$28,000"
2,"$120,000"
3,"$450,000"
4,"$75,000"
5,"$62,000"
6,"$81,000"
7,"$60,000"


In [32]:
## can't do numeric calculations as a text type
# customers.Income.median()

In [33]:
# remove the punctuation
customers.Income.str.replace('$', '').str.replace(',', '')

0     45000 
1     28000 
2    120000 
3    450000 
4     75000 
5     62000 
6     81000 
7     60000 
Name: Income, dtype: object

In [34]:
# convert it to a numeric field
pd.to_numeric(customers.Income.str.replace('$', '').str.replace(',', ''))

0     45000
1     28000
2    120000
3    450000
4     75000
5     62000
6     81000
7     60000
Name: Income, dtype: int64

In [35]:
# save the update to the dataframe
customers.Income = pd.to_numeric(customers.Income.str.replace('$', '').str.replace(',', ''))

In [36]:
# view the dataframe
customers

,Name,Age,Followers,Income,Sign Up Date,Discount,Education Level
0,Aria,25.0,0.0,45000,5/18/23,Yes,College
1,Chord,19.0,12.0,28000,8/23/23,Yes,High School
2,Harmony,26.0,0.0,120000,4/25/23,No,Graduate School
3,Melody,47.0,0.0,450000,5/5/23,No,College
4,Reed,52.0,0.0,75000,6/14/23,Yes,High School
5,Selena,30.0,1.0,62000,8/26/23,No,College
6,Stefani,30.0,0.0,81000,9/24/23,No,College
7,Taylor,33.0,52.0,60000,9/8/23,No,High School


In [37]:
# view the data types
customers.dtypes

Name                object
Age                float64
Followers          float64
Income               int64
Sign Up Date        object
Discount            object
Education Level     object
dtype: object

In [38]:
# optional: change age and followers to int types as well
customers.Age = customers.Age.astype(int)
customers.Followers = customers.Followers.astype(int)

In [39]:
# view the dataframe
customers

,Name,Age,Followers,Income,Sign Up Date,Discount,Education Level
0,Aria,25,0,45000,5/18/23,Yes,College
1,Chord,19,12,28000,8/23/23,Yes,High School
2,Harmony,26,0,120000,4/25/23,No,Graduate School
3,Melody,47,0,450000,5/5/23,No,College
4,Reed,52,0,75000,6/14/23,Yes,High School
5,Selena,30,1,62000,8/26/23,No,College
6,Stefani,30,0,81000,9/24/23,No,College
7,Taylor,33,52,60000,9/8/23,No,High School


In [40]:
# view the data types
customers.dtypes

Name               object
Age                 int64
Followers           int64
Income              int64
Sign Up Date       object
Discount           object
Education Level    object
dtype: object

#### ii. Converting to DateTime

In [41]:
# sign up date was read in as a text field
customers[['Sign Up Date']]

,Sign Up Date
0,5/18/23
1,8/23/23
2,4/25/23
3,5/5/23
4,6/14/23
5,8/26/23
6,9/24/23
7,9/8/23


In [42]:
# convert it to a datetime field
pd.to_datetime(customers['Sign Up Date'], format='%m/%d/%y')

0   2023-05-18
1   2023-08-23
2   2023-04-25
3   2023-05-05
4   2023-06-14
5   2023-08-26
6   2023-09-24
7   2023-09-08
Name: Sign Up Date, dtype: datetime64[ns]

In [43]:
# save the update to the clean dataframe
customers['Sign Up Date'] = pd.to_datetime(customers['Sign Up Date'], format='%m/%d/%y')

In [44]:
# check the dataframe
customers

,Name,Age,Followers,Income,Sign Up Date,Discount,Education Level
0,Aria,25,0,45000,2023-05-18,Yes,College
1,Chord,19,12,28000,2023-08-23,Yes,High School
2,Harmony,26,0,120000,2023-04-25,No,Graduate School
3,Melody,47,0,450000,2023-05-05,No,College
4,Reed,52,0,75000,2023-06-14,Yes,High School
5,Selena,30,1,62000,2023-08-26,No,College
6,Stefani,30,0,81000,2023-09-24,No,College
7,Taylor,33,52,60000,2023-09-08,No,High School


In [45]:
# check the data types
customers.dtypes

Name                       object
Age                         int64
Followers                   int64
Income                      int64
Sign Up Date       datetime64[ns]
Discount                   object
Education Level            object
dtype: object

#### iii. Extracting DateTime Components

In [46]:
# extract the month
customers['Sign Up Date'].dt.month

0    5
1    8
2    4
3    5
4    6
5    8
6    9
7    9
Name: Sign Up Date, dtype: int32

In [47]:
# save it as a new column
customers['Sign Up Month'] = customers['Sign Up Date'].dt.month
customers

,Name,Age,Followers,Income,Sign Up Date,Discount,Education Level,Sign Up Month
0,Aria,25,0,45000,2023-05-18,Yes,College,5
1,Chord,19,12,28000,2023-08-23,Yes,High School,8
2,Harmony,26,0,120000,2023-04-25,No,Graduate School,4
3,Melody,47,0,450000,2023-05-05,No,College,5
4,Reed,52,0,75000,2023-06-14,Yes,High School,6
5,Selena,30,1,62000,2023-08-26,No,College,8
6,Stefani,30,0,81000,2023-09-24,No,College,9
7,Taylor,33,52,60000,2023-09-08,No,High School,9


In [48]:
# extract the day of the week (0 = monday)
customers['Sign Up Date'].dt.dayofweek

0    3
1    2
2    1
3    4
4    2
5    5
6    6
7    4
Name: Sign Up Date, dtype: int32

In [49]:
# save it as a new column
customers['Sign Up DOW'] = customers['Sign Up Date'].dt.dayofweek
customers

,Name,Age,Followers,Income,Sign Up Date,Discount,Education Level,Sign Up Month,Sign Up DOW
0,Aria,25,0,45000,2023-05-18,Yes,College,5,3
1,Chord,19,12,28000,2023-08-23,Yes,High School,8,2
2,Harmony,26,0,120000,2023-04-25,No,Graduate School,4,1
3,Melody,47,0,450000,2023-05-05,No,College,5,4
4,Reed,52,0,75000,2023-06-14,Yes,High School,6,2
5,Selena,30,1,62000,2023-08-26,No,College,8,5
6,Stefani,30,0,81000,2023-09-24,No,College,9,6
7,Taylor,33,52,60000,2023-09-08,No,High School,9,4


In [50]:
# delete the sign up date column
customers = customers.drop(columns=['Sign Up Date'])
customers

,Name,Age,Followers,Income,Discount,Education Level,Sign Up Month,Sign Up DOW
0,Aria,25,0,45000,Yes,College,5,3
1,Chord,19,12,28000,Yes,High School,8,2
2,Harmony,26,0,120000,No,Graduate School,4,1
3,Melody,47,0,450000,No,College,5,4
4,Reed,52,0,75000,Yes,High School,6,2
5,Selena,30,1,62000,No,College,8,5
6,Stefani,30,0,81000,No,College,9,6
7,Taylor,33,52,60000,No,High School,9,4


### c. Conditional Logic

In [51]:
customers[['Discount']]

,Discount
0,Yes
1,Yes
2,No
3,No
4,Yes
5,No
6,No
7,No


In [52]:
# turn the field into 1 and 0 values
import numpy as np

np.where(customers['Discount'] == 'Yes', 1, 0)

array([1, 1, 0, 0, 1, 0, 0, 0])

In [53]:
# save the values back into the column
customers['Discount'] = np.where(customers['Discount'] == 'Yes', 1, 0)
customers

,Name,Age,Followers,Income,Discount,Education Level,Sign Up Month,Sign Up DOW
0,Aria,25,0,45000,1,College,5,3
1,Chord,19,12,28000,1,High School,8,2
2,Harmony,26,0,120000,0,Graduate School,4,1
3,Melody,47,0,450000,0,College,5,4
4,Reed,52,0,75000,1,High School,6,2
5,Selena,30,1,62000,0,College,8,5
6,Stefani,30,0,81000,0,College,9,6
7,Taylor,33,52,60000,0,High School,9,4


### d. Dummy Variables

In [54]:
# view a categorical field
customers[['Education Level']]

,Education Level
0,College
1,High School
2,Graduate School
3,College
4,High School
5,College
6,College
7,High School


In [55]:
# create dummy variables
pd.get_dummies(customers['Education Level'])

,College,Graduate School,High School
0,True,False,False
1,False,False,True
2,False,True,False
3,True,False,False
4,False,False,True
5,True,False,False
6,True,False,False
7,False,False,True


In [56]:
# optional: drop one of the dummy columns (for regression)
pd.get_dummies(customers['Education Level'], drop_first=True)

,Graduate School,High School
0,False,False
1,False,True
2,True,False
3,False,False
4,False,True
5,False,False
6,False,False
7,False,True


In [57]:
# make them 1s and 0s
pd.get_dummies(customers['Education Level']).astype(int)

,College,Graduate School,High School
0,1,0,0
1,0,0,1
2,0,1,0
3,1,0,0
4,0,0,1
5,1,0,0
6,1,0,0
7,0,0,1


In [58]:
# save the output
dummies_edu = pd.get_dummies(customers['Education Level']).astype(int)
dummies_edu

,College,Graduate School,High School
0,1,0,0
1,0,0,1
2,0,1,0
3,1,0,0
4,0,0,1
5,1,0,0
6,1,0,0
7,0,0,1


In [59]:
# view the customers table again
customers

,Name,Age,Followers,Income,Discount,Education Level,Sign Up Month,Sign Up DOW
0,Aria,25,0,45000,1,College,5,3
1,Chord,19,12,28000,1,High School,8,2
2,Harmony,26,0,120000,0,Graduate School,4,1
3,Melody,47,0,450000,0,College,5,4
4,Reed,52,0,75000,1,High School,6,2
5,Selena,30,1,62000,0,College,8,5
6,Stefani,30,0,81000,0,College,9,6
7,Taylor,33,52,60000,0,High School,9,4


In [60]:
# add on the dummy variables
pd.concat([customers, dummies_edu], axis=1)

,Name,Age,Followers,Income,Discount,Education Level,Sign Up Month,Sign Up DOW,College,Graduate School,High School
0,Aria,25,0,45000,1,College,5,3,1,0,0
1,Chord,19,12,28000,1,High School,8,2,0,0,1
2,Harmony,26,0,120000,0,Graduate School,4,1,0,1,0
3,Melody,47,0,450000,0,College,5,4,1,0,0
4,Reed,52,0,75000,1,High School,6,2,0,0,1
5,Selena,30,1,62000,0,College,8,5,1,0,0
6,Stefani,30,0,81000,0,College,9,6,1,0,0
7,Taylor,33,52,60000,0,High School,9,4,0,0,1


In [61]:
# save the dataframe
customers = pd.concat([customers, dummies_edu], axis=1)
customers

,Name,Age,Followers,Income,Discount,Education Level,Sign Up Month,Sign Up DOW,College,Graduate School,High School
0,Aria,25,0,45000,1,College,5,3,1,0,0
1,Chord,19,12,28000,1,High School,8,2,0,0,1
2,Harmony,26,0,120000,0,Graduate School,4,1,0,1,0
3,Melody,47,0,450000,0,College,5,4,1,0,0
4,Reed,52,0,75000,1,High School,6,2,0,0,1
5,Selena,30,1,62000,0,College,8,5,1,0,0
6,Stefani,30,0,81000,0,College,9,6,1,0,0
7,Taylor,33,52,60000,0,High School,9,4,0,0,1


In [62]:
# drop the current categorical column
customers = customers.drop(columns=['Education Level'])
customers

,Name,Age,Followers,Income,Discount,Sign Up Month,Sign Up DOW,College,Graduate School,High School
0,Aria,25,0,45000,1,5,3,1,0,0
1,Chord,19,12,28000,1,8,2,0,0,1
2,Harmony,26,0,120000,0,4,1,0,1,0
3,Melody,47,0,450000,0,5,4,1,0,0
4,Reed,52,0,75000,1,6,2,0,0,1
5,Selena,30,1,62000,0,8,5,1,0,0
6,Stefani,30,0,81000,0,9,6,1,0,0
7,Taylor,33,52,60000,0,9,4,0,0,1


## 3. Feature Engineering

In [63]:
# the data from setting the row granularity, with a few more customers
songs_genres_dict = {'Customer': ['Aria', 'Chord', 'Harmony', 'Melody', 'Reed', 'Selena', 'Stefani', 'Taylor'],
                     '# Songs': [99, 51, 18, 2, 5, 60, 15, 121],
                     'Indie': [48, 36, 5, 0, 0, 20, 2, 19],
                     'Pop': [50, 15, 10, 0, 0, 20, 5, 89],
                     'Rock': [1, 0, 3, 2, 5, 20, 8, 13]}

songs_genres = pd.DataFrame(songs_genres_dict)
songs_genres

,Customer,# Songs,Indie,Pop,Rock
0,Aria,99,48,50,1
1,Chord,51,36,15,0
2,Harmony,18,5,10,3
3,Melody,2,0,0,2
4,Reed,5,0,0,5
5,Selena,60,20,20,20
6,Stefani,15,2,5,8
7,Taylor,121,19,89,13


In [64]:
# the data from making the fields non-null and numeric
customers

,Name,Age,Followers,Income,Discount,Sign Up Month,Sign Up DOW,College,Graduate School,High School
0,Aria,25,0,45000,1,5,3,1,0,0
1,Chord,19,12,28000,1,8,2,0,0,1
2,Harmony,26,0,120000,0,4,1,0,1,0
3,Melody,47,0,450000,0,5,4,1,0,0
4,Reed,52,0,75000,1,6,2,0,0,1
5,Selena,30,1,62000,0,8,5,1,0,0
6,Stefani,30,0,81000,0,9,6,1,0,0
7,Taylor,33,52,60000,0,9,4,0,0,1


In [65]:
# combine them into a single table for modeling
model_df = pd.concat([customers, songs_genres], axis=1).drop(columns=['Customer'])
model_df

,Name,Age,Followers,Income,Discount,Sign Up Month,Sign Up DOW,College,Graduate School,High School,# Songs,Indie,Pop,Rock
0,Aria,25,0,45000,1,5,3,1,0,0,99,48,50,1
1,Chord,19,12,28000,1,8,2,0,0,1,51,36,15,0
2,Harmony,26,0,120000,0,4,1,0,1,0,18,5,10,3
3,Melody,47,0,450000,0,5,4,1,0,0,2,0,0,2
4,Reed,52,0,75000,1,6,2,0,0,1,5,0,0,5
5,Selena,30,1,62000,0,8,5,1,0,0,60,20,20,20
6,Stefani,30,0,81000,0,9,6,1,0,0,15,2,5,8
7,Taylor,33,52,60000,0,9,4,0,0,1,121,19,89,13


### a. Applying Calculations

In [66]:
# view the dataframe again
model_df

,Name,Age,Followers,Income,Discount,Sign Up Month,Sign Up DOW,College,Graduate School,High School,# Songs,Indie,Pop,Rock
0,Aria,25,0,45000,1,5,3,1,0,0,99,48,50,1
1,Chord,19,12,28000,1,8,2,0,0,1,51,36,15,0
2,Harmony,26,0,120000,0,4,1,0,1,0,18,5,10,3
3,Melody,47,0,450000,0,5,4,1,0,0,2,0,0,2
4,Reed,52,0,75000,1,6,2,0,0,1,5,0,0,5
5,Selena,30,1,62000,0,8,5,1,0,0,60,20,20,20
6,Stefani,30,0,81000,0,9,6,1,0,0,15,2,5,8
7,Taylor,33,52,60000,0,9,4,0,0,1,121,19,89,13


In [67]:
# create a new column for % pop
model_df['Pct_Pop'] = model_df['Pop'] / model_df['# Songs']
model_df

,Name,Age,Followers,Income,Discount,Sign Up Month,Sign Up DOW,College,Graduate School,High School,# Songs,Indie,Pop,Rock,Pct_Pop
0,Aria,25,0,45000,1,5,3,1,0,0,99,48,50,1,0.505051
1,Chord,19,12,28000,1,8,2,0,0,1,51,36,15,0,0.294118
2,Harmony,26,0,120000,0,4,1,0,1,0,18,5,10,3,0.555556
3,Melody,47,0,450000,0,5,4,1,0,0,2,0,0,2,0.000000
4,Reed,52,0,75000,1,6,2,0,0,1,5,0,0,5,0.000000
5,Selena,30,1,62000,0,8,5,1,0,0,60,20,20,20,0.333333
6,Stefani,30,0,81000,0,9,6,1,0,0,15,2,5,8,0.333333
7,Taylor,33,52,60000,0,9,4,0,0,1,121,19,89,13,0.735537


### b. Binning Values

In [68]:
# view the day of week column
model_df[['Name', 'Sign Up DOW']]

,Name,Sign Up DOW
0,Aria,3
1,Chord,2
2,Harmony,1
3,Melody,4
4,Reed,2
5,Selena,5
6,Stefani,6
7,Taylor,4


In [69]:
# create an indicator for weekend or not
model_df['Weekend'] = np.where(customers['Sign Up DOW'].isin([5, 6]), 1, 0)
model_df[['Name', 'Sign Up DOW', 'Weekend']]

,Name,Sign Up DOW,Weekend
0,Aria,3,0
1,Chord,2,0
2,Harmony,1,0
3,Melody,4,0
4,Reed,2,0
5,Selena,5,1
6,Stefani,6,1
7,Taylor,4,0


In [70]:
# drop the dow column
model_df = model_df.drop(columns=['Sign Up DOW'])
model_df

,Name,Age,Followers,Income,Discount,Sign Up Month,College,Graduate School,High School,# Songs,Indie,Pop,Rock,Pct_Pop,Weekend
0,Aria,25,0,45000,1,5,1,0,0,99,48,50,1,0.505051,0
1,Chord,19,12,28000,1,8,0,0,1,51,36,15,0,0.294118,0
2,Harmony,26,0,120000,0,4,0,1,0,18,5,10,3,0.555556,0
3,Melody,47,0,450000,0,5,1,0,0,2,0,0,2,0.000000,0
4,Reed,52,0,75000,1,6,0,0,1,5,0,0,5,0.000000,0
5,Selena,30,1,62000,0,8,1,0,0,60,20,20,20,0.333333,1
6,Stefani,30,0,81000,0,9,1,0,0,15,2,5,8,0.333333,1
7,Taylor,33,52,60000,0,9,0,0,1,121,19,89,13,0.735537,0


### c. Proxy Variables

In [71]:
# view the sign up month column: larger month != better
model_df[['Name', 'Sign Up Month']]

,Name,Sign Up Month
0,Aria,5
1,Chord,8
2,Harmony,4
3,Melody,5
4,Reed,6
5,Selena,8
6,Stefani,9
7,Taylor,9


In [72]:
# external data with the average temperature in F each month in Chicago
avg_temp_dict = {'Month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                 'Avg_Temp': [32, 36, 45, 56, 66, 77, 82, 81, 74, 62, 50, 37]}

avg_temp = pd.DataFrame(avg_temp_dict)
avg_temp

,Month,Avg_Temp
0,1,32
1,2,36
2,3,45
3,4,56
4,5,66
5,6,77
6,7,82
7,8,81
8,9,74
9,10,62


In [73]:
# merge the external data with the sign up month
model_df = pd.merge(model_df, avg_temp, left_on = 'Sign Up Month', right_on = 'Month')
model_df

,Name,Age,Followers,Income,Discount,Sign Up Month,College,Graduate School,High School,# Songs,Indie,Pop,Rock,Pct_Pop,Weekend,Month,Avg_Temp
0,Aria,25,0,45000,1,5,1,0,0,99,48,50,1,0.505051,0,5,66
1,Melody,47,0,450000,0,5,1,0,0,2,0,0,2,0.000000,0,5,66
2,Chord,19,12,28000,1,8,0,0,1,51,36,15,0,0.294118,0,8,81
3,Selena,30,1,62000,0,8,1,0,0,60,20,20,20,0.333333,1,8,81
4,Harmony,26,0,120000,0,4,0,1,0,18,5,10,3,0.555556,0,4,56
5,Reed,52,0,75000,1,6,0,0,1,5,0,0,5,0.000000,0,6,77
6,Stefani,30,0,81000,0,9,1,0,0,15,2,5,8,0.333333,1,9,74
7,Taylor,33,52,60000,0,9,0,0,1,121,19,89,13,0.735537,0,9,74


In [74]:
# drop the sign up month column
model_df = model_df.drop(columns=['Sign Up Month'])
model_df

,Name,Age,Followers,Income,Discount,College,Graduate School,High School,# Songs,Indie,Pop,Rock,Pct_Pop,Weekend,Month,Avg_Temp
0,Aria,25,0,45000,1,1,0,0,99,48,50,1,0.505051,0,5,66
1,Melody,47,0,450000,0,1,0,0,2,0,0,2,0.000000,0,5,66
2,Chord,19,12,28000,1,0,0,1,51,36,15,0,0.294118,0,8,81
3,Selena,30,1,62000,0,1,0,0,60,20,20,20,0.333333,1,8,81
4,Harmony,26,0,120000,0,0,1,0,18,5,10,3,0.555556,0,4,56
5,Reed,52,0,75000,1,0,0,1,5,0,0,5,0.000000,0,6,77
6,Stefani,30,0,81000,0,1,0,0,15,2,5,8,0.333333,1,9,74
7,Taylor,33,52,60000,0,0,0,1,121,19,89,13,0.735537,0,9,74


## 4. Feature Selection

### a. Exclude ID Columns

In [75]:
# remember the name values
names = model_df['Name']
names

0       Aria
1     Melody
2      Chord
3     Selena
4    Harmony
5       Reed
6    Stefani
7     Taylor
Name: Name, dtype: object

In [76]:
# exclude the column from modeling
model_df = model_df.drop(columns=['Name'])
model_df

,Age,Followers,Income,Discount,College,Graduate School,High School,# Songs,Indie,Pop,Rock,Pct_Pop,Weekend,Month,Avg_Temp
0,25,0,45000,1,1,0,0,99,48,50,1,0.505051,0,5,66
1,47,0,450000,0,1,0,0,2,0,0,2,0.000000,0,5,66
2,19,12,28000,1,0,0,1,51,36,15,0,0.294118,0,8,81
3,30,1,62000,0,1,0,0,60,20,20,20,0.333333,1,8,81
4,26,0,120000,0,0,1,0,18,5,10,3,0.555556,0,4,56
5,52,0,75000,1,0,0,1,5,0,0,5,0.000000,0,6,77
6,30,0,81000,0,1,0,0,15,2,5,8,0.333333,1,9,74
7,33,52,60000,0,0,0,1,121,19,89,13,0.735537,0,9,74


### b. Select a Subset of Features

In [77]:
# start with these columns for modeling
model_df_subset = model_df[['Age', '# Songs', 'Pct_Pop']]
model_df_subset

,Age,# Songs,Pct_Pop
0,25,99,0.505051
1,47,2,0.000000
2,19,51,0.294118
3,30,60,0.333333
4,26,18,0.555556
5,52,5,0.000000
6,30,15,0.333333
7,33,121,0.735537


In [78]:
# keep the names values in mind
names

0       Aria
1     Melody
2      Chord
3     Selena
4    Harmony
5       Reed
6    Stefani
7     Taylor
Name: Name, dtype: object

## 5. Feature Scaling

In [79]:
# view the dataframe for modeling
model_df_subset

,Age,# Songs,Pct_Pop
0,25,99,0.505051
1,47,2,0.000000
2,19,51,0.294118
3,30,60,0.333333
4,26,18,0.555556
5,52,5,0.000000
6,30,15,0.333333
7,33,121,0.735537


### a. Normalization

In [80]:
# import the library
from sklearn.preprocessing import MinMaxScaler

In [81]:
# create an object
mm_scaler = MinMaxScaler()

In [82]:
# fit the data
mm_scaler.fit(model_df_subset)

MinMaxScaler()

In [83]:
# view the range that was calculated
mm_scaler.data_min_, mm_scaler.data_max_

(array([19.,  2.,  0.]), array([ 52.        , 121.        ,   0.73553719]))

In [84]:
# apply the transformation
mm_scaler.transform(model_df_subset)

array([[0.18181818, 0.81512605, 0.6866417 ],
       [0.84848485, 0.        , 0.        ],
       [0.        , 0.41176471, 0.39986781],
       [0.33333333, 0.48739496, 0.45318352],
       [0.21212121, 0.13445378, 0.75530587],
       [1.        , 0.02521008, 0.        ],
       [0.33333333, 0.1092437 , 0.45318352],
       [0.42424242, 1.        , 1.        ]])

In [85]:
# can also do the fit and transform all in one step
normalized = mm_scaler.fit_transform(model_df_subset)
normalized

array([[0.18181818, 0.81512605, 0.6866417 ],
       [0.84848485, 0.        , 0.        ],
       [0.        , 0.41176471, 0.39986781],
       [0.33333333, 0.48739496, 0.45318352],
       [0.21212121, 0.13445378, 0.75530587],
       [1.        , 0.02521008, 0.        ],
       [0.33333333, 0.1092437 , 0.45318352],
       [0.42424242, 1.        , 1.        ]])

In [86]:
# normalization code in one code block
from sklearn.preprocessing import MinMaxScaler

mm_scaler = MinMaxScaler()
normalized = mm_scaler.fit_transform(model_df_subset)
pd.DataFrame(normalized, columns=model_df_subset.columns)

,Age,# Songs,Pct_Pop
0,0.181818,0.815126,0.686642
1,0.848485,0.000000,0.000000
2,0.000000,0.411765,0.399868
3,0.333333,0.487395,0.453184
4,0.212121,0.134454,0.755306
5,1.000000,0.025210,0.000000
6,0.333333,0.109244,0.453184
7,0.424242,1.000000,1.000000


In [87]:
# view the range of the data
df_norm = pd.DataFrame(normalized, columns=model_df_subset.columns)
df_norm.describe()

,Age,# Songs,Pct_Pop
count,8.000000,8.000000,8.000000
mean,0.416667,0.372899,0.468523
std,0.340440,0.376023,0.349796
min,0.000000,0.000000,0.000000
25%,0.204545,0.088235,0.299901
50%,0.333333,0.273109,0.453184
75%,0.530303,0.569328,0.703808
max,1.000000,1.000000,1.000000


### b. Standardization

In [88]:
# import the library
from sklearn.preprocessing import StandardScaler

In [89]:
# create an object
std_scaler = StandardScaler()

In [90]:
# fit the data
std_scaler.fit(model_df_subset)

StandardScaler()

In [91]:
# view the stats that were calculated
std_scaler.mean_, std_scaler.var_

(array([32.75      , 46.375     ,  0.34461595]),
 array([1.10437500e+02, 1.75198438e+03, 5.79223949e-02]))

In [92]:
# apply the transformation
std_scaler.transform(model_df_subset)

array([[-0.73746841,  1.25726549,  0.66661437],
       [ 1.35599029, -1.06016449, -1.4318981 ],
       [-1.30841169,  0.11049602, -0.20982319],
       [-0.26168234,  0.32551529, -0.04687987],
       [-0.64231119, -0.677908  ,  0.87646562],
       [ 1.83177636, -0.98849139, -1.4318981 ],
       [-0.26168234, -0.74958109, -0.04687987],
       [ 0.0237893 ,  1.78286816,  1.62429915]])

In [93]:
# can also do the fit and transform all in one step
standardized = std_scaler.fit_transform(model_df_subset)
standardized

array([[-0.73746841,  1.25726549,  0.66661437],
       [ 1.35599029, -1.06016449, -1.4318981 ],
       [-1.30841169,  0.11049602, -0.20982319],
       [-0.26168234,  0.32551529, -0.04687987],
       [-0.64231119, -0.677908  ,  0.87646562],
       [ 1.83177636, -0.98849139, -1.4318981 ],
       [-0.26168234, -0.74958109, -0.04687987],
       [ 0.0237893 ,  1.78286816,  1.62429915]])

In [94]:
# standardization code in one code block
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
standardized = std_scaler.fit_transform(model_df_subset)
pd.DataFrame(standardized, columns=model_df_subset.columns)

,Age,# Songs,Pct_Pop
0,-0.737468,1.257265,0.666614
1,1.355990,-1.060164,-1.431898
2,-1.308412,0.110496,-0.209823
3,-0.261682,0.325515,-0.046880
4,-0.642311,-0.677908,0.876466
5,1.831776,-0.988491,-1.431898
6,-0.261682,-0.749581,-0.046880
7,0.023789,1.782868,1.624299


In [95]:
# view the mean and standard deviation of the data
df_std = pd.DataFrame(standardized, columns=model_df_subset.columns)
df_std.describe()

,Age,# Songs,Pct_Pop
count,8.000000e+00,8.000000,8.000000
mean,-1.387779e-17,0.000000,0.000000
std,1.069045e+00,1.069045,1.069045
min,-1.308412e+00,-1.060164,-1.431898
25%,-6.661005e-01,-0.809309,-0.515342
50%,-2.616823e-01,-0.283706,-0.046880
75%,3.568396e-01,0.558453,0.719077
max,1.831776e+00,1.782868,1.624299
